# Automated Two-Stage Dividend Growth Model using Multiple CF Types

### In this advanced finance tutorial in Python, learn about:
* Two-Stage Dividend Growth Model using different cashflow types:
    * Free Cash Flow per share
    * Earnings per share
    * Dividends per share
* Calculating the Discount Rate with the Capital Asset Pricing Model (CAPM)
* Calculating Risk-Free Rate of Return, Market Rate of Return, Market Risk Premium
* Estimating Long-Term Revenue and Earnings Growth Rates
* How to estimate the Fair Value Price per share of stock based on the two-stage model
* How to use Classes, Methods, Functions, Exception-Handling, and Dataframes in Python

#### This is an advanced tutorial. For previous tutorials to fill in any gaps in learning, please check:
* [How to Fetch Stock Data](https://github.com/yusifrefae/Jupyter-Projects/blob/main/fetch-stock-data.ipynb)
* [Stock Options Data Analysis](https://github.com/yusifrefae/Jupyter-Projects/blob/main/stock-options-data-analysis.ipynb)
* [Calculate Expected Move using Options](https://github.com/yusifrefae/Jupyter-Projects/blob/main/calculate-expected-move.ipynb)

### Let's Get Started! First we'll import libraries and create a static function to fetch stock data from Yahoo

In [1]:
#Import necessary libraries
import pandas as pd
import yfinance as yf
import numpy as np
import datetime

def get_yf(value, symbol):
    """ Fetch a specific value for a given stock symbol using yfinance. Return 0 if the value is None or missing"""
    try:
        x = yf.Ticker(symbol).info.get(value)
        if x is None:
            return 0
        else:
            return x
    except Exception as e:
        return 0 

### Notice the try-except block. This is there to return 0, in case the attribute isn't in the data we pull from Yahoo
* For example, let's iterate to get the beta and dividend rate of a few stocks
* Notice that ETFs like SPY (S&P 500) and TLT (Long-Term US Treasuries) return zeroes instead of None

In [2]:
for stock in ["PFE", "GIS", "TSLA", "NVDA", "SPY", "TLT"]:
    print(f"Beta of {stock}: {get_yf('beta', stock)}")
    print(f"Dividend of {stock}: ${get_yf('dividendRate', stock)}")
    print()

Beta of PFE: 0.465
Dividend of PFE: $1.72

Beta of GIS: -0.025
Dividend of GIS: $2.44

Beta of TSLA: 2.086
Dividend of TSLA: $0

Beta of NVDA: 2.123
Dividend of NVDA: $0.04

Beta of SPY: 0
Dividend of SPY: $0

Beta of TLT: 0
Dividend of TLT: $0



#### Notice that SPY and TLT returned beta and dividend of 0.
* But if we call them directly using the ticker we get no output.
* The try-except block handles that, and simply returns 0 if the value isn't present


In [3]:
yf.Ticker("TLT").info.get('dividendRate')

### Next, let's estimate the risk-free rate using the iShares ticker SGOV
* The short-term return on US government bonds is considered to be the **risk-free** rate of return
* In reality no investment is risk-free
* But we use short-term treasuries to approximate the risk-free rate
* Because it's unlikely the US government will ever default on short-term obligations like 

#### Let's define another static function, which uses our user-defined function get_yf to get SGOV's dividend yield
* Notice that the function doesn't take any inputs, not even a ticker symbol, since the risk-free rate is independent of any given stock
* It's not a perfect estimate. It's only rounded to two decimals. But it will do for our purposes.

In [4]:
def get_rf_rate():
    """Estimate of risk-free rate that uses iShares 0-3 month Treasury dividend yield (Ticker: SGOV)"""
    return float(get_yf('dividendYield', 'SGOV') / 100)

In [5]:
print(f"The risk-free rate is estimated to be: {get_rf_rate() * 100:.4f}%")

The risk-free rate is estimated to be: 4.3500%


### Now, before getting into calculating the market return, let's discuss the capital asset pricing model (CAPM) formula

## $$ \text{Discount Rate} = R_f + \beta \times (R_m - R_f) $$

#### Rf = Risk-Free Rate
#### Rm = Return on the Market
#### B = The Stock's Beta
#### Discount Rate is the investors' required rate of return on the stock


* CAPM uses the risk-free rate and expected return on the market
* It also uses a stock's beta, which is a proxy for how volatile the stock's earnings are relative to the market's earnings
* Think of the term after the addition sign as a risk premium for a particular stock (scaled by its beta) **above** the risk-free rate

* The discount rate is the rate we'll use to discount the stock's expected cashflows later, in the two-stage growth model
* For now, let's focus on calculating the discount rate.
* We've already seen how to get beta and the risk-free rate. 

### Now let's focus on estimating Rm, the return on the market, so we can calculate the Discount Rate


#### First, we get the maximum period of daily data for the S&P500 (Ticker: SPY)

In [6]:
spy = yf.Ticker("SPY").history(period='max')
spy

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
1993-01-29 00:00:00-05:00,24.330336,24.330336,24.209289,24.313044,1003200,0.0,0.0,0.0
1993-02-01 00:00:00-05:00,24.330340,24.485971,24.330340,24.485971,480500,0.0,0.0,0.0
1993-02-02 00:00:00-05:00,24.468665,24.555127,24.416788,24.537834,201300,0.0,0.0,0.0
1993-02-03 00:00:00-05:00,24.572429,24.814522,24.555137,24.797230,529400,0.0,0.0,0.0
1993-02-04 00:00:00-05:00,24.883684,24.952853,24.607006,24.900976,531500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-09-29 00:00:00-04:00,664.359985,665.280029,661.859985,663.679993,73499000,0.0,0.0,0.0
2025-09-30 00:00:00-04:00,662.929993,666.650024,661.609985,666.179993,86288000,0.0,0.0,0.0
2025-10-01 00:00:00-04:00,663.169983,669.369995,663.059998,668.450012,72545400,0.0,0.0,0.0


#### The data only goes back to 1993 but that's enough for our purposes
#### Next, we'll add a 'Daily Return' column that calculates the percentage return for each day based on close price

In [7]:
spy['Daily Return'] = spy['Close'].pct_change()
spy.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Daily Return
Date,,,,,,,,,
1993-01-29 00:00:00-05:00,24.330336,24.330336,24.209289,24.313044,1003200,0.0,0.0,0.0,NaN
1993-02-01 00:00:00-05:00,24.330340,24.485971,24.330340,24.485971,480500,0.0,0.0,0.0,0.007113
1993-02-02 00:00:00-05:00,24.468665,24.555127,24.416788,24.537834,201300,0.0,0.0,0.0,0.002118
1993-02-03 00:00:00-05:00,24.572429,24.814522,24.555137,24.797230,529400,0.0,0.0,0.0,0.010571
1993-02-04 00:00:00-05:00,24.883684,24.952853,24.607006,24.900976,531500,0.0,0.0,0.0,0.004184


#### Let's drop the first row since the daily return is not a number (NaN)

In [8]:
spy = spy.dropna() 
spy.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Daily Return
Date,,,,,,,,,
1993-02-01 00:00:00-05:00,24.330340,24.485971,24.330340,24.485971,480500,0.0,0.0,0.0,0.007113
1993-02-02 00:00:00-05:00,24.468665,24.555127,24.416788,24.537834,201300,0.0,0.0,0.0,0.002118
1993-02-03 00:00:00-05:00,24.572429,24.814522,24.555137,24.797230,529400,0.0,0.0,0.0,0.010571
1993-02-04 00:00:00-05:00,24.883684,24.952853,24.607006,24.900976,531500,0.0,0.0,0.0,0.004184
1993-02-05 00:00:00-05:00,24.883675,24.935552,24.745336,24.883675,492100,0.0,0.0,0.0,-0.000695


#### Just to clarify how the percentage change is calculated let's take an example from Feb 2, 1993
#### The formula is: 
## $$ \text{Daily ROI} = \frac{\text{Price Today}}{\text{Price Yesterday}} - 1 $$




#### Let's copy/paste the raw values and compare it to the column's output for that day

In [9]:
24.537842 / 24.485960 - 1

0.002118846882050107

* The output in the column is 0.002119, which is a rounded version of our result above
* So now we know the column is doing the right calculations

## Now let's find the cumulative return of the 'Daily Return' column
#### This is the total compounded return over the entire period
#### So it looks like this: 1.007113 x 1.002119 x 1.010571.... etc

In [10]:
cumulative_return = (1 + spy['Daily Return']).prod()
cumulative_return

np.float64(27.52473253196015)

#### Next, let's calculate the total years of the data for the period we're looking at

In [11]:
total_days = float((spy.index[-1] - spy.index[0]).days)
total_years = total_days / 365.25
total_years

32.6652977412731

#### Wow! The cumulative return over the last 32 years is more than 2,750%  
#### In other words, 1,000 USD invested in 1993 would be worth over 27,500 USD today.
#### TIME IN the Market Beats TIMING the Market!!

## Finally, let's use the compound annual growth rate formula to annualize the return

## $$ \text{CAGR} = \left( \frac{\text{Ending Value}}{\text{Beginning Value}} \right)^{\frac{1}{\text{Years}}} - 1 $$

## Another way to express it is:
## $$ \text{CAGR} = \left( \frac{\text{Ending Value}}{\text{Beginning Value}} \right)^{\frac{365.25}{\text{Days}}} - 1 $$

In [12]:
(cumulative_return)**(365.25/total_days) - 1

np.float64(0.10681491618175798)

In [13]:
(cumulative_return ** (1 / total_years)) - 1

np.float64(0.10681491618175798)

#### As you can see, the formulas yield the same result
## Interpretation: The S&P 500 has returned roughly 10.68% per year over the last 32.67 years
## We can now use this value as a proxy for Rm (return on the market) in our CAPM formula!
#### Here's the full code for the static function 

In [14]:
def get_market_return():
    """Calculate annualized market return (CAGR) from SPY daily close prices"""
    # Add daily return column for SPY in order to estimate Market Return
    spy = yf.Ticker("SPY").history(period='max')
    spy['Daily Return'] = spy['Close'].pct_change()
    
    spy = spy.dropna() # Drop row one

    # Multiply all daily growth factors to get total compounded return over the entire period
    cumulative_return = (1 + spy['Daily Return']).prod()
    
    # Calculate total number of years for the period
    total_days = float((spy.index[-1] - spy.index[0]).days)
    total_years = total_days / 365.25
    
    # Calculate compounded annual growth rate. This is the market return over the period.
    cagr = (cumulative_return ** (1 / total_years)) - 1
    return  cagr

In [15]:
print(f'The approximate annual return on the S&P 500 is: {get_market_return() * 100:.4f}%')

The approximate annual return on the S&P 500 is: 10.6815%


### Now let's analyze an individual stock. We will: 
* Create a class for our stock object
* Add some attributes to it
* Calculate its discount rate

### First thing we'll do is add some attributes using the get_yf function we defined at the beginning of this tutorial

In [16]:
class Stock:
    """Initialize the class with a stock symbol"""
    def __init__(self, symbol):
        self.symbol = symbol
        self.price = get_yf('currentPrice', symbol)
        self.beta = get_yf('beta', symbol)

In [17]:
myStock = Stock("PFE")

print(f"Symbol: {myStock.symbol}")
print(f"Price: {myStock.price}")
print(f"Beta: {myStock.beta}")

Symbol: PFE
Price: 27.37
Beta: 0.465


## Let's add the CAPM elements to the class initialization

In [18]:
class Stock:
    """Initialize the class with a stock symbol"""
    def __init__(self, symbol, Rf, Rm):
        self.symbol = symbol
        self.price = get_yf('currentPrice', symbol)
        self.beta = get_yf('beta', symbol)
        
        # Calculate stock's discount rate using CAPM formula
        self.mrp = Rm - Rf
        self.discount_rate = Rf + self.beta * self.mrp

In [19]:
# We pass Rf and Rm to the class because adding them to the class itself slow performance significantly
Rf = get_rf_rate()
Rm = get_market_return()  
myStock = Stock("PFE", Rf, Rm)

print(f"Symbol: {myStock.symbol}")
print(f"Price: ${myStock.price}")
print(f"Beta: {myStock.beta}")
print(f"Risk-Free Rate: {Rf * 100: .2f}%")
print(f"Return on Market: {Rm * 100: .2f}%")
print(f"Market Risk Premium: {myStock.mrp * 100: .2f}%")
print(f"Discount Rate: {myStock.discount_rate * 100: .2f}%")

Symbol: PFE
Price: $27.37
Beta: 0.465
Risk-Free Rate:  4.35%
Return on Market:  10.68%
Market Risk Premium:  6.33%
Discount Rate:  7.29%


### We can iterate to quickly get these stats for multiple tickers

In [20]:
Rf = get_rf_rate()
Rm = get_market_return()

for symbol in ["PFE", "GIS", "TSLA", "NVDA"]:
    myStock = Stock(symbol, Rf, Rm)
    
    print(f"Symbol: {myStock.symbol}")
    print(f"Price: ${myStock.price}")
    print(f"Beta: {myStock.beta}")
    print(f"Risk-Free Rate: {Rf * 100: .2f}%")
    print(f"Return on Market: {Rm * 100: .2f}%")
    print(f"Market Risk Premium: {myStock.mrp * 100: .2f}%")
    print(f"Discount Rate: {myStock.discount_rate * 100: .2f}%")
    print()

Symbol: PFE
Price: $27.37
Beta: 0.465
Risk-Free Rate:  4.35%
Return on Market:  10.68%
Market Risk Premium:  6.33%
Discount Rate:  7.29%

Symbol: GIS
Price: $50.36
Beta: -0.025
Risk-Free Rate:  4.35%
Return on Market:  10.68%
Market Risk Premium:  6.33%
Discount Rate:  4.19%

Symbol: TSLA
Price: $429.83
Beta: 2.086
Risk-Free Rate:  4.35%
Return on Market:  10.68%
Market Risk Premium:  6.33%
Discount Rate:  17.56%

Symbol: NVDA
Price: $187.62
Beta: 2.123
Risk-Free Rate:  4.35%
Return on Market:  10.68%
Market Risk Premium:  6.33%
Discount Rate:  17.79%



#### As we would expect, the discount rate is higher for more volatile tech companies than for for consumer and pharma
* Tech companies' revenues and earnings are more volatile and thus the stocks are riskier and the discount rate is higher
* Consumer goods and pharmaceutical  companies have smoother earnings and demand and are less risky, so the discount rate is lower
* So far our data analysis lines up well with economic and finance theory

### Now that we have our class created, we can iterate to get multiple discount rates across sectors

In [21]:
for symbol in ["GIS", "PFE", "TSLA", "NVDA", "BTI", "PM", "MCD", "SBUX", "CVX", "XOM", 
               "VZ", "T", "TGT", "WMT", "PEP", "KO", "IBM", "MSFT", "AMZN", "BAC", "JPM"]:
    
    myStock = Stock(symbol, Rf, Rm)
    print(f"Discount Rate for {symbol}: {myStock.discount_rate * 100: .2f}%")

Discount Rate for GIS:  4.19%
Discount Rate for PFE:  7.29%
Discount Rate for TSLA:  17.56%
Discount Rate for NVDA:  17.79%
Discount Rate for BTI:  5.49%
Discount Rate for PM:  7.38%
Discount Rate for MCD:  7.54%
Discount Rate for SBUX:  10.67%
Discount Rate for CVX:  9.50%
Discount Rate for XOM:  7.41%
Discount Rate for VZ:  6.67%
Discount Rate for T:  8.30%
Discount Rate for TGT:  11.80%
Discount Rate for WMT:  8.60%
Discount Rate for PEP:  7.25%
Discount Rate for KO:  7.03%
Discount Rate for IBM:  8.93%
Discount Rate for MSFT:  10.83%
Discount Rate for AMZN:  12.46%
Discount Rate for BAC:  12.80%
Discount Rate for JPM:  11.49%


* Again, we can see that consumer goods are the lowest (GIS is food, BTI is cigarettes)
* The discount rate for McDonald's is about the same as it is for Philip Morris, which may be an interesting commentary on fast food!
* Bank stocks (BAC, JPM) are riskier than most sectors and almost as risky as emerging tech
* Oil companies (CVX, XOM), Restaurants (MCD, SBUX), and Retail (TGT, WMT) have higher discount rates due to volatility, but not as high as tech

# Two-Stage Dividend Growth Model

* The **Two-Stage Dividend Growth Model** values a stock based on future dividend payments.
* It assumes dividends will grow at different rates during two distinct periods.
* This model is especially useful for companies expected to experience different growth rates at different stages of their life cycle.

### Components:
* **Stage 1 (High Growth Period)**: The company’s dividends grow at a high rate for the first period.
* **Stage 2 (Stable Growth Period)**: After the initial high-growth period, dividends are assumed to grow at a lower, stable rate indefinitely.

### Formula:

The price of the stock (P) is calculated as the sum of the present value of dividends during the high-growth stage and the present value of dividends in the stable-growth stage.

$$ P_0 = \frac{D_0 (1 + g_1)}{(1 + r)^1}
+ \frac{D_0 (1 + g_1)^2}{(1 + r)^2}
+ \dots
+ \frac{D_0 (1 + g_1)^t}{(1 + r)^t}
+ \text{Terminal Value} $$

Where:
* $P_0 $ = Current stock price
* $D_t$ = Dividend in year t
* $r$ = Discount rate (required rate of return)
* $g_1$ = Growth rate during the high-growth period
* $g_2$ = Stable growth rate after the high-growth period
* $t$ = Number of years in the high-growth period
$$ \text{Terminal Value} = \frac{D_0 (1 + g_1)^t (1 + g_2)}{(r - g_2)} \times \frac{1}{(1 + r)^t} $$

### Advantages:
- More realistic for companies with different growth phases.
- Allows for a more nuanced valuation compared to a constant growth model.

### Disadvantages:
- Relies on accurate growth rate projections.
- Sensitive to changes in the discount rate and growth assumptions.


### In simple terms: 
* The model takes next year's dividend as an initial input.
* The model can also use the **forward EPS** or **forward FCF** instead of the forward dividend
* It increases the cashflow according to the growth rate and number of high growth years
* These forecasted cashflows are then discounted back to year zero (using the discount rate we calculated earlier)
* The terminal value is the formula for a perpetuity, discounted back to year zero
* Adding the discounted cashflows with the discounted terminal value gives the net present value of expected future cashflows
* This output is the model's estimate of the stock's present value (i.e. price per share) today

### We're ready to take on the two-stage growth model. We need to add some more attributes to the class.

In [22]:
class Stock:
    """Initialize the class with a stock symbol"""
    def __init__(self, symbol, Rf, Rm):
        self.symbol = symbol
        self.price = get_yf('currentPrice', symbol)
        self.beta = get_yf('beta', symbol)
        
        # Calculate stock's discount rate using CAPM formula
        self.mrp = Rm - Rf
        self.discount_rate = Rf + self.beta * self.mrp

        # Calculate free cashflow per share (this is the year 0 value), prevent division by zero
        fcf = get_yf('freeCashflow', symbol)
        shares = get_yf('sharesOutstanding', symbol)
        self.fcf_0 = fcf / shares if shares else 0

        self.eps = get_yf('forwardEps', symbol) 
        self.div = get_yf('dividendRate', symbol)

### We're going to calculate the price per share using all three cashflow types: FCF, EPS, DIV, because why not?!
* Since FCF divides by shares we need to prevent division by zero with an if statement

In [23]:
print(Stock("PFE", Rf, Rm).fcf_0) 
print(Stock("PFE", Rf, Rm).eps) 
print(Stock("PFE", Rf, Rm).div) 

2.8173173155352327
2.93
1.72


#### We've added forward FCF, forward EPS, and forward Dividend per share to the class using the yfinance library
* For Forward Free Cashflow per share we get free cashflow, divide by shares
* This gives us FCF per share at year 0. We'll adjust it forward by one year later
* Notice that the Dividend per share is significantly lower than the EPS and FCF

### Before diving into the model, let's create a 'stats' dataframe to see all this data in one place, for reference
* We'll accomplish this by adding a 'get_stats' method to the class

In [24]:
class Stock:
    """Initialize the class with a stock symbol"""
    def __init__(self, symbol, Rf, Rm):
        self.symbol = symbol
        self.price = get_yf('currentPrice', symbol)
        self.beta = get_yf('beta', symbol)
        
        # Calculate stock's discount rate using CAPM formula
        self.mrp = Rm - Rf
        self.discount_rate = Rf + self.beta * self.mrp

        # Calculate free cashflow per share (this is the year 0 value), prevent division by zero
        fcf = get_yf('freeCashflow', symbol)
        shares = get_yf('sharesOutstanding', symbol)
        self.fcf_0 = fcf / shares if shares else 0

        self.eps = get_yf('forwardEps', symbol) 
        self.div = get_yf('dividendRate', symbol)
        
    def get_stats(self):
        """Return a dataframe with key stats necessary to calculate the two-stage growth model"""
        df_stats = pd.DataFrame({
            'Symbol': [self.symbol],
            'FCF Yr 0': [self.fcf_0], 
            'Fwd EPS': [self.eps],
            'Fwd Div': [self.div],
            'Beta': [self.beta],
            'Mkt Return': [Rm],
            'Risk-Free Rate': [Rf],                   
            'Mkt Risk Prem': [self.mrp],
            'Discount Rate': [self.discount_rate]})
        return df_stats

In [25]:
Stock("PFE", Rf, Rm).get_stats()

,Symbol,FCF Yr 0,Fwd EPS,Fwd Div,Beta,Mkt Return,Risk-Free Rate,Mkt Risk Prem,Discount Rate
0,PFE,2.817317,2.93,1.72,0.465,0.106815,0.0435,0.063315,0.072941


### We can iterate to add multiple rows (e.g. stocks) to the dataframe for any further analysis we'd like to do

In [26]:
def get_multiple_stats(symbols):
    """Return CAPM and Two-Stage Growth Model stats for multiple stocks"""
    all_stats = []

    # Iterate to append stats to the dataframe
    for symbol in symbols:
        df = Stock(symbol, Rf, Rm).get_stats()
        all_stats.append(df) 

    # Concatenate all the dataframes together
    stats_df = pd.concat(all_stats, ignore_index=True)
    return stats_df

In [27]:
get_multiple_stats(["GIS", "PFE", "TSLA", "NVDA", "BAC", "JPM"])

,Symbol,FCF Yr 0,Fwd EPS,Fwd Div,Beta,Mkt Return,Risk-Free Rate,Mkt Risk Prem,Discount Rate
0,GIS,3.982161,4.70,2.44,-0.025,0.106815,0.0435,0.063315,0.041917
1,PFE,2.817317,2.93,1.72,0.465,0.106815,0.0435,0.063315,0.072941
2,TSLA,0.420545,3.24,0.00,2.086,0.106815,0.0435,0.063315,0.175575
3,NVDA,2.153725,4.12,0.04,2.123,0.106815,0.0435,0.063315,0.177918
4,BAC,0.000000,3.66,1.12,1.334,0.106815,0.0435,0.063315,0.127962
5,JPM,0.000000,16.74,6.00,1.127,0.106815,0.0435,0.063315,0.114856


# OK, we have all the inputs. Time to use the model to predict stock prices! 
#### First, let's find the current year and create a list of years, based on user input

In [28]:
# Define user inputs
years = 3
myStock = Stock("PFE", Rf, Rm)
g_hi = 0.03
g_lo = 0.015

current_year = datetime.datetime.now().year
year_list = [current_year + i for i in range(years + 1)]

print(f"It's currently {current_year}")
print()
print(f"The user chose {years} years of initial high-growth")
print(f"So we'll be forecasting cashflows for the following years: {year_list}")

It's currently 2025

The user chose 3 years of initial high-growth
So we'll be forecasting cashflows for the following years: [2025, 2026, 2027, 2028]


### Now the fun part. Let's project cashflows into the future. We'll start using the dividend but later we'll use all three.
* PFE (Pfizer) is a cyclical, multi-national pharmaceutical company
* Let's be conservative and choose 3% for the high-growth phase and 1.5% as the low-growth phase (into perpetuity)
* We'll cover esimating potential growth rates later in this tutorial using data analysis
* But for now let's choose the growth rates without analysis

In [29]:
# Create list of cashflows, compounded annually by the high-growth rate
cf = myStock.div
cf_list = [cf * (1 + g_hi) ** year 
           for year in range(years + 1)]
cf_list

[1.72, 1.7716, 1.8247479999999998, 1.8794904399999999]

* Translation: For the first 3 years the dividend will grow 3%
* At this rate, by 2028 it will have grown from 1.72 to 1.88

### Now let's calculate the undiscounted terminal value. This is the value of the perpetuity in 2028.
* In other words, we take 1.88, multiply it by 1.015 (since our low-growth rate is 1.5%) and divide by: (discount rate minus low growth rate)

In [30]:
# Terminal Value Formula: (Final Dividend * (1 + Low Growth Rate)) / (Discount Rate - Low Growth Rate)
tv = ((cf_list[years] * (1 + g_lo)) / (myStock.discount_rate - g_lo))
tv

np.float64(32.9243229597949)

#### Let's double check the math

In [31]:
myStock.discount_rate

np.float64(0.07294144344075172)

In [2]:
 1.87949044 * 1.015 / (0.07294144344075172 - 0.015)

32.92432295979491

### OK so we've calculated the undiscounted stream of cashflows and terminal value. 
### Now let's add the terminal value to the last year

In [33]:
# Add terminal value to the final cashflow
cf_list[years] = cf_list[years] + tv
cf_list

[1.72, 1.7716, 1.8247479999999998, np.float64(34.8038133997949)]

#### Notice that the final cashflow is approximately 34.80 (= 1.88 + 32.92)
## Now we'll discount all cashflows back to year 0

In [34]:
# Create list of discounted cashflows based on year and discount rate we previously calculated with CAPM
pv_list = [cf_list[year] / ((1 + myStock.discount_rate) ** year) 
           for year in range(years + 1)]
pv_list

[np.float64(1.72),
 np.float64(1.6511618698582113),
 np.float64(1.5850787909730606),
 np.float64(28.177260657506853)]

### Finally, we'll put the results in a dataframe

In [35]:
df_model = pd.DataFrame({'Year': year_list,
                         'Cashflow': cf_list,
                         'Present Value': pv_list})
df_model

,Year,Cashflow,Present Value
0,2025,1.720000,1.720000
1,2026,1.771600,1.651162
2,2027,1.824748,1.585079
3,2028,34.803813,28.177261


### And calculate the NPV (sum of the present value column) to predict the stock price

In [36]:
npv = sum(df_model['Present Value'])
npv

33.133501318338126

### And compare to the current market price

In [37]:
myStock.price

27.37

### The stock appears a bit undervalued according to this single analysis

# Here's the code for the class method

In [38]:
class Stock:
    """Initialize the class with a stock symbol"""
    def __init__(self, symbol, Rf, Rm):
        self.symbol = symbol
        self.price = get_yf('currentPrice', symbol)
        self.beta = get_yf('beta', symbol)
        
        # Calculate stock's discount rate using CAPM formula
        self.mrp = Rm - Rf
        self.discount_rate = Rf + self.beta * self.mrp

        # Calculate free cashflow per share (this is the year 0 value), prevent division by zero
        fcf = get_yf('freeCashflow', symbol)
        shares = get_yf('sharesOutstanding', symbol)
        self.fcf_0 = fcf / shares if shares else 0
        
        self.eps = get_yf('forwardEps', symbol) 
        self.div = get_yf('dividendRate', symbol)
    
    def get_two_stage_model(self, cf, years, g_hi, g_lo):
        """Return a dataframe with the two-stage growth model discounted cashflow calculations.
           Years should be the number of high-growth years expected, before growth slows down.
           Also, return the NPV, which is the estimated stock price, as a second output."""
        # Get the current year. Create list of years and dataframe for two-stage growth model
        current_year = datetime.datetime.now().year
        year_list = [current_year + i for i in range(years + 1)]

        # Create list of cashflows, compounded annually by the high-growth rate
        cf_list = [cf * (1 + g_hi) ** year 
                   for year in range(years + 1)]
        
        # Terminal Value Formula: (Final Dividend * (1 + Low Growth Rate)) / (Discount Rate - Low Growth Rate)
        tv = ((cf_list[years] * (1 + g_lo)) / (self.discount_rate - g_lo))
        
        # Calculate terminal value and add it to the final cashflow
        cf_list[years] = cf_list[years] + tv
        
        # Create list of discounted cashflows based on year and discount rate we previously calculated with CAPM
        pv_list = [cf_list[year] / ((1 + self.discount_rate) ** year) 
                   for year in range(years + 1)]
        
        # Build the DataFrame
        df_model = pd.DataFrame({'Year': year_list,
                                 'Cashflow': cf_list,
                                 'Present Value': pv_list})
        
        # Return the dataframe and the NPV in a tuple
        npv = sum(df_model['Present Value'])
        return (df_model, npv)

    def get_all_models(self, years, g_hi, g_lo):
        """Print all three models and their NPV outputs, using FCF, EPS, and DIV"""
        # Iterate over all three cashflow types
        for name, cf in zip(['DIV', 'EPS', 'FCF'], [self.div, self.eps, self.fcf_0 * (1 + g_hi)]):
            df, npv = self.get_two_stage_model(cf, years, g_hi, g_lo)

            print(name + ':')
            print(df)
            print()
            print(f"Predicted Stock Price: ${npv: .2f}")
            print()
            
        # Print current market price, for reference to the models
        print()
        print(f"Current Stock Price: ${self.price: .2f}") 

In [39]:
myStock = Stock("PFE", Rf, Rm)
years = 3
g_hi = 0.03
g_lo = 0.015

df, npv = myStock.get_two_stage_model(myStock.div, years, g_hi, g_lo)
print(f"{df} \n \nPredicted Stock Price: ${npv: .2f}")

   Year   Cashflow  Present Value
0  2025   1.720000       1.720000
1  2026   1.771600       1.651162
2  2027   1.824748       1.585079
3  2028  34.803813      28.177261 
 
Predicted Stock Price: $ 33.13


### We can quickly use EPS or FCF to do the same analysis, which is super awesome.

In [40]:
df, npv = myStock.get_two_stage_model(myStock.eps, years, g_hi, g_lo)
print(f"{df} \n \nPredicted Stock Price: ${npv: .2f}")

   Year   Cashflow  Present Value
0  2025   2.930000       2.930000
1  2026   3.017900       2.812735
2  2027   3.108437       2.700163
3  2028  59.287891      47.999636 
 
Predicted Stock Price: $ 56.44


### Notice we have to multiply FCF by 1 plus the high-growth rate to make it **forward** FCF per share

In [41]:
fcf = myStock.fcf_0 * (1 + g_hi)

df, npv = myStock.get_two_stage_model(fcf, years, g_hi, g_lo)
print(f"{df} \n \nPredicted Stock Price: ${npv: .2f}")

   Year   Cashflow  Present Value
0  2025   2.901837       2.901837
1  2026   2.988892       2.785699
2  2027   3.078559       2.674209
3  2028  58.718016      47.538263 
 
Predicted Stock Price: $ 55.90


### Notice that the estimates using FCF and EPS are much higher than the esimate using Dividends

## We can also use the 'get all models' method to return all three models in one command

In [42]:
myStock.get_all_models(years, g_hi, g_lo)

DIV:
   Year   Cashflow  Present Value
0  2025   1.720000       1.720000
1  2026   1.771600       1.651162
2  2027   1.824748       1.585079
3  2028  34.803813      28.177261

Predicted Stock Price: $ 33.13

EPS:
   Year   Cashflow  Present Value
0  2025   2.930000       2.930000
1  2026   3.017900       2.812735
2  2027   3.108437       2.700163
3  2028  59.287891      47.999636

Predicted Stock Price: $ 56.44

FCF:
   Year   Cashflow  Present Value
0  2025   2.901837       2.901837
1  2026   2.988892       2.785699
2  2027   3.078559       2.674209
3  2028  58.718016      47.538263

Predicted Stock Price: $ 55.90


Current Stock Price: $ 27.37


### We can also get predictions for multiple symbols

In [43]:
for symbol in ["GIS", "PFE", "TSLA", "NVDA", "BAC", "JPM"]:
    stock = Stock(symbol, Rf, Rm)
    fcf = stock.fcf_0 * (1 + g_hi)
    
    df, npv = stock.get_two_stage_model(stock.div, years, g_hi, g_lo)

    print(symbol + ":")
    print(f"DIV Prediction: ${npv: .2f}")

    df, npv = stock.get_two_stage_model(stock.eps, years, g_hi, g_lo)
    print(f"EPS Prediction: ${npv: .2f}")

    df, npv = stock.get_two_stage_model(fcf, years, g_hi, g_lo)
    print(f"FCF Prediction: ${npv: .2f}")    
    print()

GIS:
DIV Prediction: $ 98.48
EPS Prediction: $ 189.70
FCF Prediction: $ 165.55

PFE:
DIV Prediction: $ 33.13
EPS Prediction: $ 56.44
FCF Prediction: $ 55.90

TSLA:
DIV Prediction: $ 0.00
EPS Prediction: $ 24.52
FCF Prediction: $ 3.28

NVDA:
DIV Prediction: $ 0.30
EPS Prediction: $ 30.79
FCF Prediction: $ 16.58

BAC:
DIV Prediction: $ 11.59
EPS Prediction: $ 37.88
FCF Prediction: $ 0.00

JPM:
DIV Prediction: $ 69.49
EPS Prediction: $ 193.88
FCF Prediction: $ 0.00



* Of course these results are meaningless, since the growth rates are all the same, which is unrealistic
* For example, it's highly unlikely Tesla or Nvidia would grow at only 1.5% to 3% per year! They should grow faster.
* Likewise, General Mills (GIS) should probably grow slower than that. For example the market is pricing in -1% to 1% growth rates

In [44]:
stock = Stock("GIS", Rf, Rm)
df, npv = stock.get_two_stage_model(stock.div, years, 0.01, -0.01)
print(df)
print()
print(f"{stock.symbol} Predicted Price: ${npv: .2f}")
print(f"{stock.symbol} Current Price: ${stock.price: .2f}")

   Year   Cashflow  Present Value
0  2025   2.440000       2.440000
1  2026   2.464400       2.365255
2  2027   2.489044       2.292800
3  2028  50.451778      44.604322

GIS Predicted Price: $ 51.70
GIS Current Price: $ 50.36


# So now, let's explore how to estimate growth rates using financial data
* First, we'll pull the income statement
* It only has a few years of data
* But it's enough to estimate intermediate-term growth rates

In [45]:
yf.Ticker('PFE').income_stmt

,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,-9.399600e+08,-8.721300e+08,-2.938560e+08,2.264800e+07,NaN
Tax Rate For Calcs,2.100000e-01,2.100000e-01,9.600000e-02,7.600000e-02,NaN
Normalized EBITDA,2.260300e+10,1.371000e+10,4.409200e+10,3.049500e+10,NaN
Total Unusual Items,-4.476000e+09,-4.153000e+09,-3.061000e+09,2.980000e+08,NaN
Total Unusual Items Excluding Goodwill,-4.476000e+09,-4.153000e+09,-3.061000e+09,2.980000e+08,NaN
Net Income From Continuing Operation Net Minority Interest,8.020000e+09,2.134000e+09,3.136600e+10,2.241300e+10,NaN
Reconciled Depreciation,7.013000e+09,6.290000e+09,5.064000e+09,5.191000e+09,NaN
Reconciled Cost Of Revenue,1.612400e+10,2.339700e+10,3.288900e+10,2.933000e+10,NaN
EBITDA,1.812700e+10,9.557000e+09,4.103100e+10,3.079300e+10,NaN
EBIT,1.111400e+10,3.267000e+09,3.596700e+10,2.560200e+10,NaN


### Now let's define a function that returns only net income and total revenue data

In [46]:
def get_historical_tr_ni(symbol):
    """Get a stock's multi-year income statements from yfinance and 
       clean it up to show only the net income and total revenue"""
    
    df = yf.Ticker(symbol).income_stmt.loc[['Total Revenue', 'Net Income']]
    
    # Drop NaN column and transpose dataframe so that dates are rows, not columns
    df = df.dropna(axis=1, how='any').T

    # Reverse order of rows so older years are at top
    df = df[::-1]
    return df

In [47]:
get_historical_tr_ni('PFE')

,Total Revenue,Net Income
2021-12-31,8.128800e+10,2.197900e+10
2022-12-31,1.011750e+11,3.137200e+10
2023-12-31,5.955400e+10,2.119000e+09
2024-12-31,6.362700e+10,8.031000e+09


### Now we'll apply the CAGR formula we used previously on the S&P 500 
* This examples calculates the annualized growth rate for total revenue and net income but you could use EBITDA or other IS items
* As we did previously with daily return for the S&P 500, let's calculate the perecentage change, this time across years

In [48]:
df = get_historical_tr_ni('PFE')

df['RevPctChange'] = df['Total Revenue'].pct_change()
df['NIPctChange'] = df['Net Income'].pct_change()

# Drop NaN row
df = df.dropna()
df

,Total Revenue,Net Income,RevPctChange,NIPctChange
2022-12-31,1.011750e+11,3.137200e+10,0.244649,0.427362
2023-12-31,5.955400e+10,2.119000e+09,-0.411376,-0.932456
2024-12-31,6.362700e+10,8.031000e+09,0.068392,2.789995


### First we get the cumulative return, as we did before

In [49]:
cumulative_tr_return = (1 + df['RevPctChange']).prod()
cumulative_ni_return = (1 + df['NIPctChange']).prod()
cumulative_tr_return, cumulative_ni_return

(np.float64(0.7827354591083554), np.float64(0.36539423995632203))

### Now we get the number of periods to analyze (which is the same as the number of rows in the dataframe)

In [50]:
total_years = len(df) 
total_years

3

### Finally, calculate the annual return for TR and NI using the CAGR formula

In [51]:
cagr_tr = (cumulative_tr_return)**(1/total_years) - 1
cagr_ni = (cumulative_ni_return)**(1/total_years) - 1
cagr_tr, cagr_ni

(np.float64(-0.07840876391375418), np.float64(-0.28508584012162597))

### The entire function:

In [52]:
def estimate_growth_rates(symbol):
    """Estimates a stock's growth rates based on annualized growth of total revenue 
       and net income, pulled from the company's historical income statements using yfinance"""
    df = get_historical_tr_ni(symbol)

    # Display only the columns we'll need for analysis
    df['RevPctChange'] = df['Total Revenue'].pct_change()
    df['NIPctChange'] = df['Net Income'].pct_change()
    
    # Drop NaN row
    df = df.dropna()

    # Calculate the cumulative return for TR and NI over the entire period
    cumulative_tr_return = (1 + df['RevPctChange']).prod()
    cumulative_ni_return = (1 + df['NIPctChange']).prod()

    # Get total years which is the number of rows in the dataframe
    total_years = len(df) 

    # Calculate CAGR for TR and NI
    cagr_tr = (cumulative_tr_return)**(1/total_years) - 1
    cagr_ni = (cumulative_ni_return)**(1/total_years) - 1
    
    return cagr_tr, cagr_ni

In [53]:
tr_growth, ni_growth = estimate_growth_rates("PFE")
print(f"Total Revenue Growth: {tr_growth * 100: .2f}%")
print(f"Net Income Growth: {ni_growth * 100: .2f}%")   

Total Revenue Growth: -7.84%
Net Income Growth: -28.51%


#### Pfizer has experienced negative growth in the wake of its pandemic boom, these might not be realistic rates moving forward
* Let's examine GIS

In [54]:
tr_growth, ni_growth = estimate_growth_rates("GIS")
print(f"Total Revenue Growth: {tr_growth * 100: .2f}%")
print(f"Net Income Growth: {ni_growth * 100: .2f}%")   

Total Revenue Growth:  0.86%
Net Income Growth: -5.36%


### So now we have some interesting data for growth rates we can use in the two-stage model

In [55]:
years = 3
g_hi = 0.075
g_lo = -0.015
Rf = get_rf_rate()
Rm = get_market_return()

Stock('GIS', Rf, Rm).get_all_models(years, g_hi, g_lo)

DIV:
   Year   Cashflow  Present Value
0  2025   2.440000       2.440000
1  2026   2.623000       2.517475
2  2027   2.819725       2.597409
3  2028  55.488812      49.057554

Predicted Stock Price: $ 56.61

EPS:
   Year    Cashflow  Present Value
0  2025    4.700000       4.700000
1  2026    5.052500       4.849234
2  2027    5.431437       5.003207
3  2028  106.884187      94.496109

Predicted Stock Price: $ 109.05

FCF:
   Year   Cashflow  Present Value
0  2025   4.280823       4.280823
1  2026   4.601885       4.416747
2  2027   4.947026       4.556988
3  2028  97.351549      86.068322

Predicted Stock Price: $ 99.32


Current Stock Price: $ 50.36


#### The stock appears to be fairly or slightly undervalued based on the dividend model 
#### It appears to be heavily undervalued in the EPS and FCF models

#### We could continue further analyzing more stocks, pulling growth rates and using them as inputs to the model

# But wait, can't you do all this in Excel?

#### The answer is: yes, you can. BUT:
* You can't do it dynamically and this fast (you'd spend hours doing what this class does in seconds)
* Pulling API data into Excel is possible with VBA, but it's code-heavy and annoying and more complex.
* You'd have to manually do a lot of this: scraping inputs, estimating rates (believe me, I've done this, it's very labor-intensive!)

# Enjoy the Power of Python! And happy stock-picking, everybody

### For reference here is the full code

In [56]:
import pandas as pd
import yfinance as yf
import numpy as np
import datetime
from yahooquery import Ticker

def get_yf(value, symbol):
    """ Fetch a specific value for a given stock symbol using yfinance. 
        Return 0 if the value is None or missing"""
    try:
        x = yf.Ticker(symbol).info.get(value)
        if x is None:
            return 0
        else:
            return x
    except Exception as e:
        return 0       
###################################################################################################

def get_rf_rate():
    """Estimate of risk-free rate that uses iShares 
        0-3 month Treasury dividend yield (Ticker: SGOV)"""
    return float(get_yf('dividendYield', 'SGOV') / 100)
###################################################################################################

def get_market_return():
    """Calculate annualized market return (CAGR) from SPY daily close prices"""
    # Add daily return column for SPY in order to estimate Market Return
    spy = yf.Ticker("SPY").history(period='max')
    spy['Daily Return'] = spy['Close'].pct_change()
    
    spy = spy.dropna() # Drop row one

    # Multiply all daily growth factors to get total compounded return over the entire period
    cumulative_return = (1 + spy['Daily Return']).prod()
    
    # Calculate total number of years for the period
    total_days = float((spy.index[-1] - spy.index[0]).days)
    total_years = total_days / 365.25
    
    # Calculate compounded annual growth rate. This is the market return over the period.
    cagr = (cumulative_return ** (1 / total_years) - 1)
    return  cagr
###################################################################################################
    
def get_multiple_stats(symbols):
    """Return CAPM and Two-Stage Growth Model stats for multiple stocks"""
    all_stats = []

    # Iterate to append stats to the dataframe
    for symbol in symbols:
        df = Stock(symbol, Rf, Rm).get_stats()
        all_stats.append(df) 

    # Concatenate all the dataframes together
    stats_df = pd.concat(all_stats, ignore_index=True)
    return stats_df
###################################################################################################

class Stock:
    """Initialize the class with a stock symbol"""
    def __init__(self, symbol, Rf, Rm):
        self.symbol = symbol
        self.price = get_yf('currentPrice', symbol)
        self.beta = get_yf('beta', symbol)
        
        # Calculate stock's discount rate using CAPM formula
        self.mrp = Rm - Rf
        self.discount_rate = Rf + self.beta * self.mrp

        # Calculate free cashflow per share (this is the year 0 value), prevent division by zero
        fcf = get_yf('freeCashflow', symbol)
        shares = get_yf('sharesOutstanding', symbol)
        self.fcf_0 = fcf / shares if shares else 0

        self.eps = get_yf('forwardEps', symbol) 
        self.div = get_yf('dividendRate', symbol)
    ###############################################################################################
    
    def get_stats(self):
        """Return a dataframe with key stats necessary to calculate the two-stage growth model"""
        df_stats = pd.DataFrame({
            'Symbol': [self.symbol],
            'FCF Yr 0': [self.fcf_0], 
            'Fwd EPS': [self.eps],
            'Fwd Div': [self.div],
            'Beta': [self.beta],
            'Mkt Return': [Rm],
            'Risk-Free Rate': [Rf],                   
            'Mkt Risk Prem': [self.mrp],
            'Discount Rate': [self.discount_rate]})
        return df_stats
    ###############################################################################################
    
    def get_two_stage_model(self, cf, years, g_hi, g_lo):
        """Return a dataframe with the two-stage growth model discounted cashflow calculations.
           Years should be the number of high-growth years expected, before growth slows down.
           Also, return the NPV, which is the estimated stock price, as a second output."""
        # Get the current year. Create list of years and dataframe for two-stage growth model
        current_year = datetime.datetime.now().year
        year_list = [current_year + i for i in range(years + 1)]
        
        # Create list of cashflows, compounded annually by the high-growth rate
        cf_list = [cf * (1 + g_hi) ** year 
                   for year in range(years + 1)]
        
        # Terminal Value Formula: (Final Dividend * (1 + Low Growth Rate)) / (Discount Rate - Low Growth Rate)
        tv = ((cf_list[years] * (1 + g_lo)) / (self.discount_rate - g_lo))
        
        # Calculate terminal value and add it to the final cashflow
        cf_list[years] = cf_list[years] + tv
        
        # Create list of discounted cashflows based on year and discount rate we previously calculated with CAPM
        pv_list = [cf_list[year] / ((1 + self.discount_rate) ** year) 
                   for year in range(years + 1)]
        
        # Build the DataFrame
        df_model = pd.DataFrame({'Year': year_list,
                                 'Cashflow': cf_list,
                                 'Present Value': pv_list})
        
        # Return the dataframe and the NPV in a tuple
        npv = sum(df_model['Present Value'])
        return (df_model, npv)
    ###############################################################################################

    def get_all_models(self, years, g_hi, g_lo):
        """Print all three models and their NPV outputs, using FCF, EPS, and DIV"""
        # Iterate over all three cashflow types
        for name, cf in zip(['DIV', 'EPS', 'FCF'], [self.div, self.eps, self.fcf_0 * (1 + g_hi)]):
            df, npv = self.get_two_stage_model(cf, years, g_hi, g_lo)

            print(name + ':')
            print(df)
            print()
            print(f"Predicted Stock Price: ${npv: .2f}")
            
            #print(self.get_two_stage_model(cf, years, g_hi, g_lo))
            # print()
        
        # Print current market price, for reference to the models
        print()
        print(f"Current Stock Price: {self.price}") 
###################################################################################################

def get_historical_tr_ni(symbol):
    """Get a stock's multi-year income statements from yfinance and 
       clean it up to show only the net income and total revenue"""
    
    df = yf.Ticker(symbol).income_stmt.loc[['Total Revenue', 'Net Income']]
    
    # Drop NaN column and transpose dataframe so that dates are rows, not columns
    df = df.dropna(axis=1, how='any').T

    # Reverse order of rows so older years are at top
    df = df[::-1]
    return df
###################################################################################################

def estimate_growth_rates(symbol):
    """Estimates a stock's growth rates based on annualized growth of total revenue 
       and net income, pulled from the company's historical income statements using yfinance"""
    df = get_historical_tr_ni(symbol)

    # Display only the columns we'll need for analysis
    df['RevPctChange'] = df['Total Revenue'].pct_change()
    df['NIPctChange'] = df['Net Income'].pct_change()
    
    # Drop NaN row
    df = df.dropna()

    # Calculate the cumulative return for TR and NI over the entire period
    cumulative_tr_return = (1 + df['RevPctChange']).prod()
    cumulative_ni_return = (1 + df['NIPctChange']).prod()

    # Get total years which is the number of rows in the dataframe
    total_years = len(df) 

    # Calculate CAGR for TR and NI
    cagr_tr = (cumulative_tr_return)**(1/total_years) - 1
    cagr_ni = (cumulative_ni_return)**(1/total_years) - 1
    
    return cagr_tr, cagr_ni

## Author: Yusif Refae, MBA
#### Let's work together: <a href = "https://www.linkedin.com/in/yusifrefae/">Message me on LinkedIn!</a>